<h2>202140090-홍기현-Assignment3</h2>

In [1]:
# Build a simple neural network with 3 hidden layers.
# Activation function for each layer is sigmoid, and final output is produced using softmax.
# A file for reading mnist is provided. Download "mnist.py" into the working directory.
# Fill in the * marked part and try to get the final accuracy.

<h3>필요 모듈 import</h3>
<br>

In [2]:
import sys, os

import numpy as np
import pickle
from mnist import load_mnist

<h3>Activation Function 구현</h3>
<hr>
<h4>Softmax</h4>

- 여러개의 출력을 지수 함수를 적용하여 전체 비율로 나눈다.
- 이를 통하여 전체 출력의 합은 1이되며, 큰 값은 더욱 크게, 작은 값은 더욱 작게 변한다.

<br>
<h4>Sigmoid</h4>

- Binary 출력에서 0~1 사이의 값으로 변환시킨다.
- 예를 들어 0에서는 0.5를 갖으며, 음수에서는 0 ~ 0.5, 양수에서는 0.5 ~ 1 사이의 값이다.
- 이를 통하여 0.5를 기준으로 이진 분류가 가능하다.

In [3]:
def softmax(x):
    if x.ndim == 2:
        c = np.max(x)
        x = np.exp(x - c)
    x = x - np.max(x) # handle overflow
    return np.exp(x) / np.sum(np.exp(x))
                              
                              
def sigmoid(x):
    x = np.exp(-x)
    return 1 / (1 + x)


<br>
<hr>
<h3>그 외 기능 구현</h3>
<br>

<h4>get_data() 메소드</h4>

- 서버 문제로 다운로드 코드가 작동하지 않아 직접 다운 후 수행하였다.
- 학습 데이터는 사용하지 않으며, 이미 학습된 가중치로 성능을 평가하는 코드를 작성한다.

<br>
<h4>init_network() 메소드</h4>

- 미리 학습해놓은 가중치를 읽어와 network로 반환하는 메소드

<br>
<h4>predict() 메소드</h4>

- 가중치와 bias가 저장된 network와 예측할 데이터를 파라매터로 받아 예측을 수행하는 메소드
- 은닉층은 2개, 그리고 출력층으로 이루어져 있으며, 각 은닉층은 0~1 사이의 값을 반환하는 sigmoid를, 출력층은 여러 입력에서 하나의 결과를 출력하기 위해 softmax를 Activation Function으로 이용
- 각 층에서 예측을 하는 식은 <strong>결과 = (가중치 * 입력값) + 바이어스</strong>

In [4]:
def get_data():
    (x_train, t_train), (x_test, t_test) = load_mnist(normalize=True, flatten=True, one_hot_label=False)
    return x_test, t_test


def init_network():
    with open("sample_weight.pkl", 'rb') as f:
        network = pickle.load(f)
    return network


def predict(network, x):
    W1, W2, W3 = network['W1'], network['W2'], network['W3']
    b1, b2, b3 = network['b1'], network['b2'], network['b3']
    
    # 가중치를 계산하고 바이어스를 더함
    a1 = np.dot(x, W1) + b1
    # activation Function 적용
    z1 = sigmoid(a1)

    a2 = np.dot(z1, W2) + b2
    z2 = sigmoid(a2)

    a3 = np.dot(z2, W3) + b3
    y = softmax(a3)
    
    return y

<br>
<hr>
<h3>예측 수행하기</h3>

- get_data()를 통해 x에 테스트 데이터를, t에 테스트 데이터에 대한 정답을 반환받는다.
- init_network()를 통해 미리 학습한 가중치를 반환받는다.
- accuracy_cnt 변수는 정답을 맞출 때마다 1 증가시킨다.

- predict() 메소드를 통해 학습된 모델이 테스트 데이터를 어떻게 예측하는지 받는다.
- softmax 활성화 함수의 결과이므로, 이 중 최대값이 모델이 실제로 예측한 값이다. 따라서, argmax()를 적용하여 최대값을 출력한다.
- 만약, 예측값이 정답괴 일치한다면 accuracy_cnt를 1 증가시킨다.

- 최종 결과는 정답을 맞춘 정답률이다.

In [5]:
x, t = get_data()

network = init_network()
accuracy_cnt = 0

for i in range(len(x)):
    # predict을 통해 각각의 테스트 데이터의 예측값을 구함
    y = predict(network, x[i])
    # 예측값의 최대값이 모델이 예측한 결과
    p = np.argmax(y)
    
    # 만약 예측값이 정답과 같다면 Accuracy_cnt를 1증가
    if p == t[i]:
        accuracy_cnt += 1

# 전체 테스트 데이터의 개수 중 정답을 맞춤 비율을 출력
print("Accuracy:" + str(float(accuracy_cnt) / len(x)))

Accuracy:0.9352
